In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#Se cargan las reviews de Darden
path_data = '../data/clean'
data_darden = pd.read_parquet(f'{path_data}/reviews_darden.parquet')

In [53]:
data_darden.sample(3)

,name,review_id,user_id,business_id,stars,text,date,day,text_reply,city,state,postal_code,coordinates
1815,Eddie V's Prime Seafood,9Uj8OFtsOWxDOsoNa4WIIw,tPQgjnHutx_DnfxoswR89g,mc9V_wG4QyRYew9_S8R44A,5.0,I forgot to take pictures as everything was am...,2019-03-22,Friday,,Tampa,FL,33607,"27.9602024049,-82.5183134644"
19,LongHorn Steakhouse,8l-UKYS3J_oY6mm0UO2nsw,8x5hhBfAMY5eHYw1dTLBrw,TFu4tLmV6zRVLEFG0TCLBA,5.0,So we drove from Miami to come to Busch Garden...,2020-12-25,Friday,,Tampa,PA,33612,"28.0550872249,-82.4363535575"
3321,Seasons 52,kV9MvH-_m3rUEUsryc3FMg,LlmHXai7HCRUSDX5u6P5Sg,mYMPepp0QIZRk_52pWzeoA,5.0,New Years Eve 2013 was most memorable for 3 co...,2014-02-20,Thursday,,Tampa,PA,33609,"27.9463699545,-82.5248121533"


In [54]:
reviews_darden_negativas = data_darden[data_darden.stars <= 3][['review_id','date','name','stars','text']]
reviews_darden_negativas.sample(3)

,review_id,date,name,stars,text
6490,rAQ8dH5xD_SgBu4LfgM7BA,2018-04-18,Olive Garden Italian Restaurant,1.0,My partner and I ate at this olive garden on 4...
6130,RTeCNbe5hiNmWCm6pZygJQ,2019-02-09,LongHorn Steakhouse,3.0,Ok place if Outback is too busy. The apps were...
2211,CA2U7UZ8nC1CoZjDPjBFxA,2019-12-27,Eddie V's Prime Seafood,3.0,My husband and I celebrating anniversary -was ...


In [3]:
with open('modelo_ML_negativas2.pkl', 'rb') as archivo:
    modelo_ml = pickle.load(archivo)

In [56]:
def df_reviews_etiquetadas(df_reviews):
    with open('modelos_ml_pkl/modelo_ML_negativas.pkl', 'rb') as archivo:
        modelo_ml = pickle.load(archivo)
    problematicas = [modelo_ml.predict([review])[0] for review in df_reviews.text]

    df_reviews_ml = pd.DataFrame({
    'review_id': df_reviews.review_id,
    'date': df_reviews.date,
    'name': df_reviews.name,
    'text': df_reviews.text,
    'problem': problematicas
    })

    df_problematicas = df_reviews_ml['problem'].apply(pd.Series)
    df_problematicas.rename(columns={0: 'bad_food', 1: 'cost', 2: 'filthy', 3: 'missing_food', 4: 'order_problem', 5: 'rude_service', 6: 'bad_neighborhood', 7: 'slow_service'}, inplace=True)
    df_reviews_negativas_etiquetadas = pd.concat([df_reviews[['review_id', 'date','name','text']], df_problematicas], axis=1)
    df_reviews_negativas_etiquetadas['date'] = pd.to_datetime(df_reviews_negativas_etiquetadas['date'])

    
    df_reviews_negativas_etiquetadas['quarter'] = df_reviews_negativas_etiquetadas['date'].dt.to_period('Q')

    # Se por trimestre y restaurante, y crear un diccionario las reseñas y sus problematicas
    df_reviews_negativas_etiquetadas_agrupadas = df_reviews_negativas_etiquetadas.groupby(['name', 'quarter']).apply(lambda x: {
        'review_id': x['review_id'].count(),
        'bad_food': sum(x['bad_food'].tolist()),
        'cost': sum(x['cost'].tolist()),
        'filthy': sum(x['filthy'].tolist()),
        'missing_food': sum(x['missing_food'].tolist()),
        'order_problem': sum(x['order_problem'].tolist()),
        'rude_service': sum(x['rude_service'].tolist()),
        'bad_neighborhood': sum(x['bad_neighborhood'].tolist()),
        'slow_service': sum(x['slow_service'].tolist())
    }).unstack()
    return df_reviews_negativas_etiquetadas_agrupadas.to_dict()


In [11]:
df_reviews_negativas_etiquetadas = df_reviews_etiquetadas(reviews_darden_negativas)

NameError: name 'df_reviews_etiquetadas' is not defined

In [88]:
df_reviews_negativas_etiquetadas

{Period('2006Q3', 'Q-DEC'): {'Bahama Breeze': {'review_id': 1,
   'bad_food': 0,
   'cost': 0,
   'filthy': 0,
   'missing_food': 0,
   'order_problem': 0,
   'rude_service': 0,
   'bad_neighborhood': 0,
   'slow_service': 0},
  "Cheddar's Scratch Kitchen": nan,
  "Eddie V's Prime Seafood": nan,
  'LongHorn Steakhouse': nan,
  'Olive Garden Italian Restaurant': {'review_id': 1,
   'bad_food': 0,
   'cost': 0,
   'filthy': 0,
   'missing_food': 0,
   'order_problem': 0,
   'rude_service': 0,
   'bad_neighborhood': 0,
   'slow_service': 0},
  "Ruth's Chris Steak House": nan,
  'Seasons 52': nan,
  'Yard House': nan},
 Period('2007Q2', 'Q-DEC'): {'Bahama Breeze': nan,
  "Cheddar's Scratch Kitchen": nan,
  "Eddie V's Prime Seafood": nan,
  'LongHorn Steakhouse': {'review_id': 1,
   'bad_food': 1,
   'cost': 0,
   'filthy': 0,
   'missing_food': 0,
   'order_problem': 0,
   'rude_service': 0,
   'bad_neighborhood': 0,
   'slow_service': 0},
  'Olive Garden Italian Restaurant': nan,
  "Ruth'